In [3]:
import sys
sys.path.append(r"C:\Users\siyam\Desktop\Legal DS\project\venv\Lib\site-packages")
import os
from pathlib import Path
import json
import random
import numpy as np
import spacy


from tqdm import tqdm
import pickle
import spacy
import json
import random
import re
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn import model_selection
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.symbols import ORTH
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
import matplotlib.pyplot as plt
from spacy.language import Language
from luima_sbd import sbd_utils as luima
import seaborn as sns
import plotly.express as px
import math
import fasttext
%matplotlib inline

# Phase 4.1: Custom Fasttext embeddings

In [4]:
tokened_data = '../Data/tokenized_sentences_unlabeled_corpus.txt'

In [5]:
tokened_data

'../Data/tokenized_sentences_unlabeled_corpus.txt'

In [6]:
import fasttext

In [40]:
%time model = fasttext.train_unsupervised(tokened_data, epoch=10, minCount=20, verbose=2)

CPU times: total: 2h 34min 8s
Wall time: 22min 53s


In [41]:
len(model.get_words(on_unicode_error='ignore'))

12217

In [42]:
model.save_model("../Data/ft_word_embedding_model.bin")

In [7]:
model = fasttext.load_model("../Data/ft_word_embedding_model.bin")
len(model.get_words(on_unicode_error='ignore'))

12217

In [44]:
# test_words=["veteran","vet","service","cause","caused","remanded","vietnam","see","board","notice","claim", "judge","records","letter"]
test_words=["veteran","v.","argues","ptsd","granted","korea","holding","also"]
test_word_track=[]
for word in test_words:
    nearest_neigh=model.get_nearest_neighbors(word, on_unicode_error='ignore')
    app_dict={
        "test_word": word,
        "nearest_neighbours": nearest_neigh
    }
    test_word_track.append(app_dict)

In [45]:
test_word_track

[{'test_word': 'veteran',
  'nearest_neighbours': [(0.743248462677002, 'appellant'),
   (0.7283560633659363, 'he'),
   (0.5777813196182251, 'additionally'),
   (0.5699048042297363, 'she'),
   (0.5583944916725159, 'have'),
   (0.5371725559234619, 'his'),
   (0.5326139330863953, 'that'),
   (0.5299844145774841, 'the'),
   (0.5259371399879456, 'furthermore'),
   (0.5086601972579956, 'moreover')]},
 {'test_word': 'v.',
  'nearest_neighbours': [(0.3488071858882904, 'mercy'),
   (0.3253656029701233, 'sulfur'),
   (0.32075878977775574, 'sulfa'),
   (0.31695881485939026, 'berry'),
   (0.3139955699443817, 'harry'),
   (0.31391584873199463, 'split'),
   (0.31163278222084045, 'retired'),
   (0.30642300844192505, 'dmd'),
   (0.30287694931030273, 'larry'),
   (0.3027796149253845, 'furnish')]},
 {'test_word': 'argues',
  'nearest_neighbours': [(0.7081536650657654, 'argue'),
   (0.6161292791366577, 'holbrook'),
   (0.5763423442840576, 'criticize'),
   (0.5669669508934021, 'arguendo'),
   (0.548126935

In [9]:
model.get_word_vector('veteran').shape

(100,)

In [13]:
model.get_word_vector('I am a man.')

array([ 0.04571989,  0.05541565,  0.01460429, -0.00451844, -0.04302831,
        0.01515509, -0.03478996, -0.0591285 , -0.01954735, -0.02213247,
       -0.0305799 , -0.04605798, -0.11795374, -0.00148901, -0.01395917,
        0.01840812, -0.02118282, -0.05107199,  0.03552783,  0.07100148,
        0.05289863, -0.01372218,  0.01974634, -0.02236735,  0.06317852,
       -0.0223579 , -0.10702348,  0.08129429, -0.02659442, -0.04014803,
        0.02694385,  0.01968933,  0.00190103, -0.03060269, -0.02989491,
       -0.06229573, -0.00335177, -0.02985294, -0.00467016,  0.00664969,
       -0.07888909,  0.04751356, -0.01701034, -0.08474612, -0.01290622,
       -0.0310165 , -0.00379519, -0.01269579,  0.00479306,  0.03842088,
        0.0115279 , -0.05321871,  0.06470627,  0.00122952, -0.03627041,
       -0.04136675, -0.00369618,  0.08480741, -0.02039044,  0.09488145,
       -0.10430087,  0.02022019, -0.01013389, -0.03488074,  0.00605017,
        0.02033514, -0.00325071,  0.0203271 ,  0.05132734, -0.07

In [12]:
model.get_word_vector('baal')

array([ 0.13368388,  0.16723448, -0.19282673, -0.22463043,  0.003288  ,
        0.07603691, -0.25268188, -0.13354711, -0.19061351, -0.21470772,
       -0.27539203, -0.01667088,  0.03097813,  0.00724745,  0.46637353,
        0.01820259, -0.23130117, -0.46049795, -0.1419974 ,  0.36595532,
        0.01585658, -0.20899983,  0.12313581, -0.2122161 , -0.0675602 ,
        0.18826407, -0.6101543 ,  0.19386323, -0.01570185, -0.20840509,
       -0.13756858,  0.05380087, -0.20889989, -0.05851411,  0.28689906,
       -0.09358108,  0.01333833, -0.16564387,  0.25764343,  0.00386833,
       -0.11494599, -0.02058365, -0.11000228,  0.02697934, -0.13233414,
       -0.1463216 , -0.04306844,  0.17780767,  0.10540187, -0.10935166,
       -0.05242671,  0.08124124, -0.01462856,  0.10538583, -0.05730271,
        0.11313155, -0.255569  ,  0.10904928, -0.3125328 ,  0.3347795 ,
       -0.2936529 ,  0.25424832,  0.27167812,  0.10287657,  0.18155251,
        0.15492368, -0.2505388 , -0.11726322, -0.25859648,  0.16